# A Jupyter Notebook to poll the daily AQI readings for a location.

Build the API query URL

In [1]:
base_url = "https://www.airnowapi.org/aq/data/"

In [2]:
from datetime import datetime
from datetime import timedelta

In [3]:
timespan = 24 #hours
now = datetime.now()
then = now - timedelta(hours=timespan)

In [4]:
endDate = now.strftime("%Y-%m-%dT%H")
startDate = then.strftime("%Y-%m-%dT%H")

In [5]:
#Constant endpoints
parameters = "parameters=OZONE,PM25,PM10,CO,NO2,SO2"
boundingBox = "BBOX=-123.146648,46.996876,-121.487713,48.204308"
dataType = "dataType=B"
format_ = "format=text/csv"
verbose = "verbose=1"
nowcastonly = "nowcastonly=1"
includeRaw = "includerawconcentrations=1"

In [27]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [9]:
query_url = base_url + "?" + "startDate=" + startDate + "&" + "endDate=" + endDate + "&" + parameters + "&" + boundingBox + "&" + dataType + "&" + format_ + "&" + verbose + "&" + nowcastonly + "&" + includeRaw + "&API_KEY=" + os.getenv("AIRNOW_API_KEY")

In [10]:
import requests

In [11]:
response = requests.get(query_url)
response = response.content

In [12]:
import pandas as pd
import io

In [13]:
colnames = ["Latitude", "Longitude", "UTC", "Pollutant", "Concentration", "Unit", "Raw_Concentration", "AQI", "Category", "Site_Name", "Site_Agency", "AQS_ID", "Full_AWS_ID"]

In [14]:
df = pd.read_csv(io.StringIO(response.decode('utf-8')), names = colnames)

In [15]:
df

,Latitude,Longitude,UTC,Pollutant,Concentration,Unit,Raw_Concentration,AQI,Category,Site_Name,Site_Agency,AQS_ID,Full_AWS_ID
0,47.213551,-123.100807,2024-07-30T23:00,PM2.5,2.3,UG/M3,2.4,13,1,Shelton-W Franklin,Washington Department of Ecology,530450007,840530450007
1,47.029301,-122.821503,2024-07-30T23:00,OZONE,12.0,PPB,13.0,11,1,Lacey-College St,Washington Department of Ecology,530670013,840530670013
2,47.029301,-122.821503,2024-07-30T23:00,PM2.5,0.0,UG/M3,-2.0,0,1,Lacey-College St,Washington Department of Ecology,530670013,840530670013
3,48.129101,-122.778900,2024-07-30T23:00,PM2.5,5.5,UG/M3,5.7,31,1,Port Townsend-San Juan,Washington Department of Ecology,530310003,840530310003
4,47.592675,-122.627397,2024-07-30T23:00,PM2.5,0.1,UG/M3,0.0,1,1,Bremerton-Spruce Ave,Washington Department of Ecology,530350007,840530350007
...,...,...,...,...,...,...,...,...,...,...,...,...,...
727,47.498535,-122.278385,2024-07-31T23:00,PM2.5,3.0,UG/M3,-999.0,17,1,Tukwila_Allentown,Washington Department of Ecology,840530330069,840530330069
728,47.281400,-122.223300,2024-07-31T23:00,PM2.5,3.5,UG/M3,-999.0,19,1,Auburn 29th St,Washington Department of Ecology,840530330047,840530330047
729,48.054315,-122.171529,2024-07-31T23:00,PM2.5,2.1,UG/M3,-999.0,12,1,Marysville-7th Ave,Washington Department of Ecology,530611007,840530611007
730,47.600863,-122.148397,2024-07-31T23:00,PM2.5,3.1,UG/M3,-999.0,17,1,Bellevue-SE 12th,Washington Department of Ecology,530330031,840530330031


In [16]:
threshold = 50

In [17]:
exceeding_df = df[df.AQI >= 50]
exceeding_df

,Latitude,Longitude,UTC,Pollutant,Concentration,Unit,Raw_Concentration,AQI,Category,Site_Name,Site_Agency,AQS_ID,Full_AWS_ID


In [18]:
if len(exceeding_df) > 0 :
    print("AQI threshold exceeded for", len(exceeding_df), "station(s)")
else:
    print("All readings okay")

All readings okay


In [32]:
import pins
board = pins.board_connect()
board.pin_write(exceeding_df, "katie.masiello@posit.co/aqi_exceeding_stations", type="csv")

Writing pin:
Name: 'katie.masiello@posit.co/aqi_exceeding_stations'
Version: 20240731T232947Z-c7bbe


Meta(title='aqi_exceeding_stations: a pinned 0 x 13 DataFrame', description=None, created='20240731T232947Z', pin_hash='c7bbe54b34fec757', file='aqi_exceeding_stations.csv', file_size=124, type='csv', api_version=1, version=VersionRaw(version='68'), tags=None, name='katie.masiello@posit.co/aqi_exceeding_stations', user={}, local={})